In [3]:
import json 
import pandas as pd
def SydGrid(file_name='./sydGrid.json'):
    '''
    "geo":{"type":"Point","coordinates":[-33.86751,151.20797]}
    all 8 acesss and read same time
    datastructures to optimize performance
    showq -q snowy| less
    master has rank 0 always
    sbatch <script_name> 
    squeue -u <username>
    module avail mpip4py//list all libs
    tell module load <lib> in slurm script
    scancel <job_id>
    '''
    syd_grid = []
    with open(file_name) as f:
            data = json.load(f)
            for val in data['features']:
                grid_data = {}
                properties = val['properties']
                grid_data['id'] = properties['id']
                num = int((grid_data['id'])%4)
                if num == 0:
                    num=4
                grid_data['cell'] = chr(int(num+64))+str(int((grid_data['id']-1)/4)+1)
                geometry = val['geometry'] 
                grid_data['xmin'] = geometry['coordinates'][0][0][0]
                grid_data['ymin'] = geometry['coordinates'][0][2][1]
                grid_data['xmax'] = geometry['coordinates'][0][1][0]
                grid_data['ymax'] = geometry['coordinates'][0][0][1]
                syd_grid.append(grid_data)
    return syd_grid
grid = SydGrid()
syd_grid = pd.DataFrame(grid)
syd_grid

,id,cell,xmin,ymin,xmax,ymax
0,1,A1,149.792551,-33.31645,150.342551,-32.81645
1,2,B1,149.792551,-33.81645,150.342551,-33.31645
2,3,C1,149.792551,-34.31645,150.342551,-33.81645
3,4,D1,149.792551,-34.81645,150.342551,-34.31645
4,5,A2,150.342551,-33.31645,150.892551,-32.81645
5,6,B2,150.342551,-33.81645,150.892551,-33.31645
6,7,C2,150.342551,-34.31645,150.892551,-33.81645
7,8,D2,150.342551,-34.81645,150.892551,-34.31645
8,9,A3,150.892551,-33.31645,151.442551,-32.81645
9,10,B3,150.892551,-33.81645,151.442551,-33.31645


In [8]:
def LangCodes(file_name='./language_codes.json'):
    lang_codes = dict()
    with open(file_name) as f:
            data = json.load(f) 
            for val in data:
                lang_codes[val['code']] = val['name']
            return lang_codes
lc = LangCodes()
print(lc)

{'fr': 'French', 'en': 'English', 'ar': 'Arabic', 'ja': 'Japanese', 'es': 'Spanish', 'de': 'German', 'it': 'Italian', 'in': 'Indonesian', 'pt': 'Portuguese', 'ko': 'Korean', 'tr': 'Turkish', 'ru': 'Russian', 'nl': 'Dutch', 'fil': 'Filipino', 'msa': 'Malay', 'zh-tw': 'Traditional Chinese', 'zh-cn': 'Simplified Chinese', 'hi': 'Hindi', 'no': 'Norwegian', 'sv': 'Swedish', 'fi': 'Finnish', 'da': 'Danish', 'pl': 'Polish', 'hu': 'Hungarian', 'fa': 'Persian', 'he': 'Hebrew', 'ur': 'Urdu', 'th': 'Thai', 'uk': 'Ukrainian', 'ca': 'Catalan', 'ga': 'Irish', 'el': 'Greek', 'eu': 'Basque', 'cs': 'Czech', 'gl': 'Galician', 'ro': 'Romanian', 'hr': 'Croatian', 'en-gb': 'British English', 'vi': 'Vietnamese', 'bn': 'Bangla', 'bg': 'Bulgarian', 'sr': 'Serbian', 'sk': 'Slovak', 'gu': 'Gujarati', 'mr': 'Marathi', 'ta': 'Tamil', 'kn': 'Kannada'}


In [2]:
import subprocess
import sys
import os

SPLIT_FILE_CHUNK_SIZE = '5000'
SPLIT_PREFIX_LENGTH = '2'

FILE_NAME = "./tinyTwitter.json"
f = open(FILE_NAME, "r") 
for lineNumber, line in enumerate(f):
    if (lineNumber%id) == rank:
        texts = []
        langs = []

        try:
            json_str = json.loads(line[:-2]) #localmachine : -3, spartan : -2

            text = json_str['doc']['text']
            texts.append(text)
            lang = json_str['doc']['metadata']['iso_language_code']
            langs.append(lang)
        except: 
            pass

f.close()


TypeError: unsupported operand type(s) for %: 'int' and 'builtin_function_or_method'

In [4]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    data = [(i+1)**2 for i in range(size)]
else:
    data = None
data = comm.scatter(data, root=0)
assert data == (rank+1)**2

In [22]:
def GetData(file_name='./tinyTwitter.json'):
    used_data = []
    with open(file_name) as f:
            data = json.load(f)
            for val in data['rows']:
                tweet_data = {}
                document = val['doc']
                meta_data = document['metadata']
                if type(document['geo'])!= type(None):
                    tweet_data['language'] = meta_data['iso_language_code']
                    tweet_data['coordinates'] = document['geo']['coordinates']
                    used_data.append(tweet_data)
    return used_data
tdata = GetData()
print(tdata)

hi
hi
[{'language': 'en', 'coordinates': [-33.86, 151.211]}, {'language': 'en', 'coordinates': [-33.86751, 151.20797]}]
